### Voor- en nadelen van "Plain SQL"-

 `+` Eenvoudig in gebruik: test je SQL string m.b.v. een SQL tool en copy-paste naar Python
- `-` Geen SQL syntax check bij de ontwikkeling --> errors duiken op at runtime
- `-` Geen integratie met Python objecten.
- `-` Code is niet porteerbaar naar andere databanken: 
  - `-` Je gebruikt software-bibliotheken die specifiek zijn voor een bepaald database-systeem (= native API)
  - `-` SQL-code volgt dialect van een bepaalde database en is dus niet noodzakelijk porteerbaar naar een andere database (vb. `SELECT TOP ...`). 

In [3]:
import pyodbc  

conn = pyodbc.connect(r'DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost\SQL2025;DATABASE=xtreme;Trusted_Connection=yes;')
cursor = conn.cursor()

**Complexere SELECT** 
   
```sql
select top 10 p.ProductID, p.ProductName, 
sum(od.Quantity*od.UnitPrice) as sales
from Product p join OrdersDetail od
on p.ProductID=od.ProductID
group by p.ProductID, p.ProductName
order by sum(od.Quantity*od.UnitPrice) desc;
```

In [5]:
stmt =  """
        select top 10 p.ProductID, p.ProductName, 
        sum(od.Quantity*od.UnitPrice) as sales
        from Product p join OrdersDetail od
        on p.ProductID=od.ProductID
        group by p.ProductID, p.ProductName
        order by sum(od.Quantity*od.UnitPrice) desc;
        """

cursor.execute(stmt)
rows = cursor.fetchall()

**UPDATE** \
Verhoog de prijzen van alle producten van het producttype x met y % (x en y zijn parameters). 

   ```sql
   update Product 
   set Price = Price * 1.10
   where ProductTypeID 
   in (select ProductTypeID 
   from Producttype where producttypename='Mountain')
   ``` 

In [ ]:
stmt = """
    update Product 
    set Price = Price * (1 + ?/100.0)
    where ProductTypeID 
    in (select ProductTypeID 
    from Producttype where producttypename=?);
    """
cursor.execute(stmt, (10, 'Mountain') )

conn.rollback()
conn.commit()

**INSERT**  
    ```sql
   insert into Courier (CourierID, CourierName,Website)
   values (11,'bpost','www.bpost.be')
   ```

In [ ]:
stmt = "insert into Courier (CourierID, CourierName,Website) values (11,'bpost','www.bpost.be');"
cursor.execute(stmt )

conn.rollback()
conn.commit()

**DELETE**
   ```sql
   delete from Courier where CourierName = 'bpost'
    ```

In [ ]:
stmt = "delete from Courier where CourierName = 'bpost';"
cursor.execute(stmt )

conn.rollback()
conn.commit()

In [ ]:
conn.close()

**Stored procedure**

```sql
select top 5 productid, productname, unitsinstock
from product
order by unitsinstock desc;
```

In [ ]:
'''
In database xtreme: 
    create or alter procedure ProductsInStock
    as
        select top 5 productid, productname, unitsinstock from product order by unitsinstock desc;
'''

stmt = "exec ProductsInStock;"
cursor.execute(stmt)
rows = cursor.fetchall()